In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('data/MetroPT3(AirCompressor).csv', index_col = False)
data.head()

# 1. Data Cleaning and Preprocessing 

According to the documentation, the following preprocessing steps have been conducted before publishing the data:

- Data segmentation
- Normalization
- Feature Extraction

Thus, we do not need to apply them in our work.

### 1) Overview 

In [ ]:
print(f'number of null values: {data.isna().sum().sum()}')

In [ ]:
print(f'number of duplicates: {data.duplicated().sum()}')

In [ ]:
print(f'shape: {data.shape}')

### 2) drop unnecessary columns:


In [ ]:
# drop unecessary columns
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

### 3) Add a label Column 
From the failure information table provided int the data description file below, we will try to label the data and evaluate the effectiveness of failure prediction algorithms: 

![alt text](image.png)

In [ ]:
labeled_data = data.copy()
labeled_data['status'] = 0

#### Converting the timestamp column into pandas.DateTime data type


In [ ]:
# converting the timestamp to datetime
labeled_data['timestamp'] = pd.to_datetime(labeled_data['timestamp'], format = '%Y-%m-%d %H:%M:%S')
print("current data type of timestamp: ", labeled_data['timestamp'].dtype)

In [ ]:
#define function to convert time to pandas.dateTime 
def convert_time(X):
    result =[]
    for x in X:
        result.append(pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S'))
    return result

failure_start_time = convert_time(["2020-04-18 00:00:00", "2020-05-29 23:30:00", "2020-06-05 10:00:00", "2020-07-15 14:30:00"])
failure_end_time = convert_time(["2020-04-18 23:59:00", "2020-05-30 06:00:00", "2020-06-07 14:30:00", "2020-07-15 19:00:00"])


In [ ]:
#iterate through the data and label the data
for start, end in zip(failure_start_time, failure_end_time):
    labeled_data.loc[(labeled_data['timestamp'] >= start) & (labeled_data['timestamp'] <= end), 'status'] = 1
    #check if any failures were missed or
    print(f"number of failures between {start} and {end}: {labeled_data.loc[(labeled_data['timestamp'] >= start) & (labeled_data['timestamp'] <= end), 'status'].sum()}")
    
print(f"number of failures: {labeled_data['status'].sum()}")

In [ ]:
#check for positive class imbalance
print(f"Example of Failure state \n {labeled_data[labeled_data['status']==1].head()}")


### 4) Undersampling

In [ ]:
labeled_data.status.value_counts()

The number of negative values (normal cases) is way too large compared to the positive class (around 30k positive samples and 1500k negative samples). Then, we are running into an Imbalaned Dataset. It is expected since we are dealing with a predictive maintenance problem.  
To address this issue, we ought to balance our data. There are various techniques to balance it. One of them is **Undersampling**. For that, we will be using the `RandomUnderSampler` from `imblearn`:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

y = labeled_data.status
X = labeled_data.iloc[:, :-1]

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

balanced_data = X_resampled
balanced_data['status'] = y_resampled

`balanced_data` is supposed to be balanced now. Let us check it:

In [ ]:
# value counts from the imbalanced dataset
imbalanced_class_counts = labeled_data['status'].value_counts()

# value counts from the balanced dataset
balanced_class_counts = balanced_data['status'].value_counts()

# plot pie charts to show the class distribution difference

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].pie(
    imbalanced_class_counts,
    labels = ['Negative', 'Positive'],
    autopct = '%1.1f%%',
    startangle = 90,
    colors = ['lightpink', 'lightblue']
)
axes[0].set_title('Before Undersampling')

axes[1].pie(
    balanced_class_counts,
    labels = ['Negative', 'Positive'],
    autopct = '%1.1f%%',
    startangle = 90,
    colors = ['lightpink', 'lightblue']
)
axes[1].set_title('After Undersampling')

plt.tight_layout()
plt.show()

In [ ]:
balanced_data.info()

### 5) Checking for outliers 


In [ ]:
def identify_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    num_outliers = len(outliers)
    print(f"Number of outliers in {column}: {num_outliers}")
    return outliers

def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_removed = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    num_removed = len(data) - len(outliers_removed)
    print(f"Number of outliers removed from {column}: {num_removed}\n")
    return outliers_removed

# First, identify outliers
clean_data = balanced_data.copy()
for col in clean_data:
    if col not in ['timestamp', 'status']:
        outliers = identify_outliers(clean_data, col)


the features: ['COMP', 'DV_eletric','Towers', 'MPG','LPS','Pressure_switch','Oil_level','Caudal_impulses'] are binary features. So we do not remove outliers.

In [ ]:
for col in clean_data:
    if col not in ['timestamp', 'status', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']:
        cleaned_data = remove_outliers(clean_data, col)

In [ ]:
#Investigate the columns with the binary values
binary_cols = ['LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']
#Ensure the the binary data is binary
cleaned_data[binary_cols] = cleaned_data[binary_cols].apply(np.round)

In [ ]:
# count the number of unique values in each column
for col in cleaned_data.columns:
    print(f"number of unique values in {col}: {cleaned_data[col].nunique()}")
    

# 2. Exploratory data analysis

### 1) Correlation

In [ ]:
# correlation 
correlation = cleaned_data.corr()
plt.figure(figsize = (10, 10))
sns.heatmap(correlation, annot = True, cmap = 'coolwarm')
plt.title('Correlation Matrix')
plt.show()


From the above correlation heatmap,  we can see that our target feature **"status"** has a strong correlation with these features: TP2, H1, DV_pressure, Oil_temparature, Motor_current, COMP, DV_electric and MPG.

### 2) Visualization

1. Outliers

In [ ]:
# visualize all the features outliers in one plot 
sns.set(rc={'figure.figsize':(20,8.27)})
sns.boxplot(data = cleaned_data.drop(['timestamp', 'status'], axis = 1))
# plt.xticks(rotation = 45)
plt.title('Boxplot of all features')
plt.show()

2. Probability distribution


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='seaborn')

#visualize the probability distribution of all the features
def plot_col_distribution(data):
    fig, axes = plt.subplots(4, 4, figsize = (20, 10))
    axes = axes.flatten()
    for i, col in enumerate(data.columns):
        data[col] = data[col].replace([np.inf, -np.inf], np.nan)
        sns.histplot(data[col], ax = axes[i], kde=True)
        axes[i].set_title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()
    
plot_col_distribution(cleaned_data.drop(['timestamp', 'status'], axis = 1))


3. Time series plot

In [ ]:
cleaned_data.iloc[:,:16]

In [ ]:
# reorganize according to timestamp 
cleaned_data.sort_values('timestamp', inplace = True)


In [ ]:
# Plot the time series
cleaned_data.iloc[:,:16].plot(
        subplots =True,
        layout=(6, 3),
        figsize=(22,22),
        fontsize=10, 
        linewidth=1,
        sharex = False, 
        title='Visualization of the Original Time Series')
plt.show()

# 3. Modeling

### 1) Train-Test Split

Before Modeling and classifying our data to be able to predict when we need maintenance, we need to split our data into train and test sets:

In [ ]:
from sklearn.model_selection import train_test_split

y = cleaned_data.status
X = cleaned_data.iloc[:, :-1].drop(columns=['timestamp'])

X_train, X_test,y_train, y_test = train_test_split(X, y,
                                   random_state=42, 
                                   test_size=0.25, 
                                   shuffle=True)

print(f'shape of X_train: {X_train.shape}')
print(f'shape of X_test: {X_test.shape}')
print(f'shape of y_train: {y_train.shape}')
print(f'shape of y_test: {y_test.shape}')

### 2) Evaluation Protocol

Reminder that the aim of this study is to predict failures and the need of maintenance in an
urban metro public transportation service. To assess the fit and how good each model performed, we will evaluate it using the following metrics:
* **Accuracy:** measures the overall correctness of the model since our data is balanced:
\begin{align*}
    Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
\end{align*}

* **Precision:** the goal is to maximize the positive (failure) predictions when they were originally failures (TP) and minimize negative calls (FN) which are failure values predicted as non-failure by the model:
\begin{align*}
    Precision = \frac{TP}{TP + FN}
\end{align*}

* **F1 Score**: harmonic mean of Precision and Recall:
\begin{align*}
    F1 Score = 2 \times \frac{Precision \times Recall}{Precision + Recall}
\end{align*}

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

We maintain a dataframe `scores` to hold the scores of each model we will study:

In [ ]:
scores = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'f1'])
scores # should be empty

### 3) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

dt_preds = dt.predict(X_test)

# accuracy
train_accuracy_dt = accuracy_score(dt.predict(X_train), y_train)
test_accuracy_dt = accuracy_score(dt_preds, y_test)

# precision
train_precision_dt = precision_score(dt.predict(X_train), y_train, average='macro')
test_precision_dt = precision_score(dt_preds, y_test, average='macro')

# f1 score
train_f1_dt = f1_score(dt.predict(X_train), y_train, average='macro')
test_f1_dt = f1_score(dt_preds, y_test, average='macro')

print(f'accuracy:')
print(f'  train: {train_accuracy_dt}')
print(f'  test: {test_accuracy_dt}')

print(f'\nprecision:')
print(f'  train: {train_precision_dt}')
print(f'  test: {test_precision_dt}')

print(f'\nf1:')
print(f'  train: {train_f1_dt}')
print(f'  test: {test_f1_dt}')

In [ ]:
# add to scores dataframe
scores.loc[len(scores)] = ['Decision Tree', test_accuracy_dt, test_precision_dt, test_f1_dt]
scores

### 4) Hyperparameterized Decision Tree

Here, we are going to find the best Decision Tree Classifier. That is, we aim to find its parameters that best maximize the score. This is done by **Random Search**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

dt_params = {
    'criterion': ['gini'],
    'max_depth': randint(1, 50),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 10)
}

random_search_dt = RandomizedSearchCV(
    estimator=dt,
    param_distributions=dt_params,
    n_iter=100,
    cv=5,
    n_jobs=1,
    random_state=42,
    scoring='f1_macro'
)

random_search_dt.fit(X_train, y_train)

best_params_dt = random_search_dt.best_params_
best_score_dt = random_search_dt.best_score_

print(f'best decision tree parameters: {best_params_dt}')
print(f'best score (F1): {best_score_dt}')

Now, we model with teh best **Decision Tree** best on the **Random Search**:

In [ ]:
best_dt = random_search_dt.best_estimator_
best_dt.fit(X_train, y_train)
best_dt_preds = best_dt.predict(X_test)

# accuracy
train_accuracy_best_dt = accuracy_score(best_dt.predict(X_train), y_train)
test_accuracy_best_dt = accuracy_score(best_dt_preds, y_test)

# precision
train_precision_best_dt = precision_score(best_dt.predict(X_train), y_train, average='macro')
test_precision_best_dt = precision_score(best_dt_preds, y_test, average='macro')

# f1 score
train_f1_best_dt = f1_score(best_dt.predict(X_train), y_train, average='macro')
test_f1_best_dt = f1_score(best_dt_preds, y_test, average='macro')

print(f'accuracy:')
print(f'  train: {train_accuracy_dt}')
print(f'  test: {test_accuracy_dt}')

print(f'\nprecision:')
print(f'  train: {train_precision_dt}')
print(f'  test: {test_precision_dt}')

print(f'\nf1:')
print(f'  train: {train_f1_dt}')
print(f'  test: {test_f1_dt}')

In [ ]:
# add to scores dataframe
scores.loc[len(scores)] = ['Hyperparametirized Decision Tree', test_accuracy_best_dt, test_precision_best_dt, test_f1_best_dt]
scores

### 5) Random Forest

### 6) K-Nearest Neighbors (KNN)

### 7) Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_preds = gnb.predict(X_test)

# accuracy
train_accuracy_gnb = accuracy_score(gnb.predict(X_train), y_train)
test_accuracy_gnb = accuracy_score(gnb_preds, y_test)

# precision
train_precision_gnb = precision_score(gnb.predict(X_train), y_train, average='macro')
test_precision_gnb = precision_score(gnb_preds, y_test, average='macro')

# f1 score
train_f1_gnb = f1_score(gnb.predict(X_train), y_train, average='macro')
test_f1_gnb = f1_score(gnb_preds, y_test, average='macro')

print(f'accuracy:')
print(f'  train: {train_accuracy_gnb}')
print(f'  test: {test_accuracy_gnb}')

print(f'\nprecision:')
print(f'  train: {train_precision_gnb}')
print(f'  test: {test_precision_gnb}')

print(f'\nf1:')
print(f'  train: {train_f1_gnb}')
print(f'  test: {test_f1_gnb}')

In [ ]:
# add to scores dataframe
scores.loc[len(scores)] = ['Gaussian Naive Bayes', test_accuracy_gnb, test_precision_gnb, test_f1_gnb]
scores

As noticeable, Gaussian Naive Bayes did perform quite well. Yet, worse than previous classifiers. This is due to the nature of the features, half of them follow a similar to Normal distribution and the rest follow a Bernoulli distribution.

### 8) Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', degree=3)
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)

# accuracy
train_accuracy_svm = accuracy_score(svm.predict(X_train), y_train)
test_accuracy_svm = accuracy_score(svm_preds, y_test)

# precision
train_precision_svm = precision_score(svm.predict(X_train), y_train, average='macro')
test_precision_svm = precision_score(svm_preds, y_test, average='macro')

# f1 score
train_f1_svm = f1_score(svm.predict(X_train), y_train, average='macro')
test_f1_svm = f1_score(svm_preds, y_test, average='macro')

print(f'accuracy:')
print(f'  train: {train_accuracy_svm}')
print(f'  test: {test_accuracy_svm}')

print(f'\nprecision:')
print(f'  train: {train_precision_svm}')
print(f'  test: {test_precision_svm}')

print(f'\nf1:')
print(f'  train: {train_f1_svm}')
print(f'  test: {test_f1_svm}')

In [ ]:
# add to scores dataframe
scores.loc[len(scores)] = ['SVM', test_accuracy_svm, test_precision_svm, test_f1_svm]
scores